# BUILD LLM Bootcamp Day 2

## Setup

### Install Libraries

In [ ]:
!pip install snowflake-ml-python==1.0.12

In [ ]:
!pip install peft transformers==4.34.0 tokenizers vllm==0.2.1.post1 bitsandbytes datasets absl-py==1.3.0

### Import Libraries

In [ ]:
from datasets import Dataset, ClassLabel
from transformers import AutoTokenizer, AutoModelForCausalLM
import sys
from utils import Concatenator
import pandas as pd
pd.set_option('display.max_colwidth', None)

import torch
from transformers import LlamaForCausalLM, LlamaTokenizer
import os
import json
from transformers import TrainerCallback
from contextlib import nullcontext
from transformers import default_data_collator, Trainer, TrainingArguments

from snowflake.snowpark.session import Session
from snowflake.snowpark import VERSION
import snowflake.snowpark.functions as F
from snowflake.ml.registry import model_registry
from snowflake.ml.model import deploy_platforms
from snowflake.ml.model.models import llm

import logging 
logger = logging.getLogger("snowflake.snowpark.session")
logger.setLevel(logging.ERROR)
logger = logging.getLogger("snowflake.ml")
logger.setLevel(logging.ERROR)

### Load Base Model and Tokenizer

In [ ]:
# TODO: Set your Hugging Face token
!huggingface-cli login --token YOUR_HG_TOKEN_GOES_HERE

In [ ]:
model_id="meta-llama/Llama-2-7b-chat-hf"
print('loading tokenizer')
tokenizer = LlamaTokenizer.from_pretrained(model_id)
print('loading model')
model =LlamaForCausalLM.from_pretrained(model_id, load_in_8bit=True, device_map='auto', torch_dtype=torch.float16)

## Prepare Datasets
### Load Datasets

*IMP NOTE: Please do NOT not run the following cell during the bootcamp session. Fine-tuning on the entire dataset could take over an hour.*

In [ ]:
# def prepare_stratified_dataset(path, seed = 42):
#     raw_df = pd.read_json(path, lines=True)
#     raw_df['id'] = raw_df.index
#     ds = Dataset.from_pandas(raw_df, split='train')
#     cl = ClassLabel(num_classes=4, names=["EN", "FR", "DE", "ES"])
#     new_features = ds.features.copy()
#     new_features['lang_label'] = cl
#     cl_d = {l : cl.str2int(l) for l in ["EN", "FR", "DE", "ES"]}
#     def convert_lang(sample):
#         sample['lang_label'] = cl_d[sample['language']]
#         return sample
#     ds = ds.map(convert_lang, features=new_features)
#     ds_split = ds.train_test_split(test_size=0.15, stratify_by_column='lang_label', seed=42)
#     test_ds_split = ds_split['test'].train_test_split(test_size=2/3, stratify_by_column='lang_label', seed=42)
#     return ds_split['train'].to_pandas(), test_ds_split['train'].to_pandas(), test_ds_split['test'].to_pandas()

# train_df, eval_df, test_df = prepare_stratified_dataset(
#     'transcripts.json'
# )

In [ ]:
df = pd.read_json("transcripts.json", lines=True)
print(f"Total records: {df.shape[0]}")
train_df = df.head(100)
print(f"Train        : {train_df.shape[0]}")
eval_df = df[200:300]
print(f"Eval         : {eval_df.shape[0]}")
test_df = df.tail(100)
print(f"Test         : {test_df.shape[0]}")

In [ ]:
train_df.head()

In [ ]:
datasets = {
    'train': Dataset.from_pandas(train_df),
    'eval': Dataset.from_pandas(eval_df),
    'test': Dataset.from_pandas(test_df)
}

### Apply Prompt to Datasets

In [ ]:
train_prompt = f"""
[INST] <<SYS>>
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
<</SYS>>
### Instruction:
{{instruction}}
### Input:
{{input_}}
### Output:
{{output}}
{{eos_token}}
"""

eval_prompt = f"""
[INST] <<SYS>>
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
<</SYS>>
### Instruction:
{{instruction}}
### Input:
{{input_}}
### Output:
"""

In [ ]:
def apply_train_template(sample):
    return {
        "text": train_prompt.format(
            instruction=sample["instruction"],
            input_=sample["input"].replace('\\n', '\n'),
            output=sample["output"],
            eos_token=tokenizer.eos_token,
        )
    }

def apply_eval_template(sample):
    return {
        "text": eval_prompt.format(
            instruction=sample["instruction"],
            input_=sample["input"].replace('\\n', '\n')
        )
    }



#applying template

datasets['train'] = datasets['train'].map(apply_train_template, remove_columns=list(datasets['train'].features))
for k in ['eval', 'test']:
    datasets[k] = datasets[k].map(apply_eval_template, remove_columns=list(datasets[k].features))


### Tokenize Datasets

In [ ]:
for k, v in datasets.items():
    datasets[k] = v.map(
        lambda sample: tokenizer(sample["text"]),
        batched=True,
        remove_columns=list(v.features),
    ).map(Concatenator(), batched=True)

## Fine-tune Llama 2 Model


Fine-tuning is one form of model training. We start training from a pre-trained model and adjust a set of model parameters to better solve for a concrete task based on task specific data. Today we are going to fine-tune 7B Llama 2 model using LoRA (Low-Rank Adaptation)--which is a parameter efficient way of fine-tuning LLM. 

Instead of adjusting all the ~7B parameters, LoRA allows us to adjust only a percent of model weights--which can save compute and memory resources dramatically. For this lab, we will fine-tune our model using LoRA on a single A10 GPU. This will demostrate how good the inference can be on fine-tuned models even with limited compute.

In [ ]:
#setting the model into training mode
model.train()

#setting up training
def create_peft_config(model):
    from peft import (
        get_peft_model,
        LoraConfig,
        TaskType,
        prepare_model_for_kbit_training,
    )

    peft_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        inference_mode=False,
        r=8,
        lora_alpha=32,
        lora_dropout=0.05,
        target_modules = ["q_proj", "v_proj"]
    )

    # prepare int-8 model for training
    model = prepare_model_for_kbit_training(model)
    model = get_peft_model(model, peft_config)
    model.print_trainable_parameters()
    return model, peft_config

# create peft config
model, lora_config = create_peft_config(model)


In [ ]:
# rm -r output_weights_dir # deletes prior fine tuning weights
!mkdir output_weights_dir

#### Configuration

To achieve good performance for the task, you will need at least 1 `num_epochs`, feel free to explore this on your own and we've set `output_weights_dir` as the directory where the fine-tuned weights will be stored after fine-tuning job completes.

In [ ]:
output_dir = "output_weights_dir"
enable_profiler = False

config = {
    'lora_config': lora_config,
    'learning_rate': 1e-4,
    'num_train_epochs': 1,
    'gradient_accumulation_steps': 2,
    'per_device_train_batch_size': 2,
    'gradient_checkpointing': False,
}

# Define training args
training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    bf16=True,  # Use BF16 if available
    # logging strategies
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=10,
    save_strategy="no",
    optim="adamw_torch_fused",
    max_steps=total_steps if enable_profiler else -1,
    **{k:v for k,v in config.items() if k != 'lora_config'}
)

### Fine-tuning

#### Configuration

We're passing `train_dataset` and `eval_dataset` that are used to generate loss calculation during fine-tuning process.

In [ ]:
profiler = nullcontext() 

with profiler:
    # Create Trainer instance
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=datasets['train'],
        eval_dataset=datasets['eval'],
        data_collator=default_data_collator,
        callbacks=[profiler_callback] if enable_profiler else [],
    )

    # Start training
    trainer.train()
    
model.save_pretrained(output_dir)

## Log and Deploy Fine-tuned Llama 2

### Establish Secure Connection

*NOTE: Update [connection.json](../connection.json) and set your password, Hugging Face token, and replace '####' with your user number.*

In [ ]:
# Create Snowflake Session object
connection_parameters = json.load(open('connection.json'))
session = Session.builder.configs(connection_parameters).create()
session.sql_simplifier_enabled = True

snowflake_environment = session.sql('select current_user(), current_version()').collect()
snowpark_version = VERSION

# Current Environment Details
print('User                        : {}'.format(snowflake_environment[0][0]))
print('Role                        : {}'.format(session.get_current_role()))
print('Database                    : {}'.format(session.get_current_database()))
print('Schema                      : {}'.format(session.get_current_schema()))
print('Warehouse                   : {}'.format(session.get_current_warehouse()))
print('Snowflake version           : {}'.format(snowflake_environment[0][1]))
print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))

### Model Registery

In [ ]:
MODEL_NAME = "LLAMA2_7b_CHAT"
MODEL_VERSION = "FineTunedV1.1"
DEPLOYMENT_NAME = "FINETUNED_LLAMA2"
MODEL_REGISTRY_DB = connection_parameters['database']
MODEL_REGISTRY_SCHEMA = connection_parameters['schema']
COMPUTE_POOL = connection_parameters['compute_pool']

registry = model_registry.ModelRegistry(
    session=session, 
    database_name=MODEL_REGISTRY_DB, 
    schema_name=MODEL_REGISTRY_SCHEMA, 
    create_if_not_exists=True)

In [ ]:
registry.list_deployments(model_name=MODEL_NAME,model_version=MODEL_VERSION).to_pandas()

In [ ]:
# registry.delete_deployment(model_name=MODEL_NAME,model_version=MODEL_VERSION,deployment_name=DEPLOYMENT_NAME)

In [ ]:
# registry.delete_model(model_name=MODEL_NAME,model_version=MODEL_VERSION,delete_artifact=True)

### Reference Llama 2 from Fine-tuning

In [ ]:
%%time
#referencing our fine tuned model
#referencing huggingface token
options = llm.LLMOptions(
    token=connection_parameters['huggingface_token'],
    max_batch_size=100,
)
#referencing our fine tuned weights and using the hugging face token to merge with base llama model
llama_model = llm.LLM(
    model_id_or_path='output_weights_dir',
    options=options
)

# log model in registry
llama_model_ref = registry.log_model(
    model_name=MODEL_NAME,
    model_version=MODEL_VERSION,
    model=llama_model
)

# deploy model
llama_model_ref.deploy(
    deployment_name=DEPLOYMENT_NAME, 
    platform=deploy_platforms.TargetPlatform.SNOWPARK_CONTAINER_SERVICES,
    permanent=True, 
   options={"compute_pool": COMPUTE_POOL, "num_gpus": 1})

In [ ]:
# if its already deployed run the following:
llama_model_ref = model_registry.ModelReference(
    registry=registry, 
    model_name=MODEL_NAME, 
    model_version=MODEL_VERSION)

## Inference on Eval Dataset using fine-tuned Llama 2

In [ ]:
eval_df['input'] = eval_df.apply(
    lambda x: eval_prompt.format(
        instruction=x["instruction"],
        input_=x["input"].replace('\\n', '\n')
    ), axis=1
)
eval_df.reset_index(drop=True, inplace=True)
eval_df.head()

In [ ]:
eval_df['predicted'] = llama_model_ref.predict(deployment_name=DEPLOYMENT_NAME,data=eval_df)#.head()
eval_df[['output', 'predicted']].head()

## Clean Up Resources

Delete deployment and the model

In [ ]:
llama_model_ref.delete_deployment(deployment_name=DEPLOYMENT_NAME)
llama_model_ref.delete_model(delete_artifact=True)